## Bayesian Optimisation for Chemical Reaction Optimisation: Example Application of BO

Well done for powering through the previous sections! Using our understanding of the basics of Bayesian Optimisation, we shall now observe an application of Bayesian Optimisation: Chemical Reaction Optimisation.

Running a reaction condition optimisation campaign can be cumbersome as experimentation can be time consuming especially under the vast combinatorial domain of the input reaction condition search space (reagent identities, equivalences, solvent identities, reaction temperature etc.). Bayesian Optimisation could be a useful method to make better decisions (balance of exploration and exploitation conditioned on known expeirmental outcomes) when deciding on which experiments to run.

Before tackling the following notebook, think about what changes to the workflow or code we have seen in previous section to allow Bayesian Optimisation to be a practical tool for experimental chemical optimisation.

### Importing relevant packages
(Feel free to import any packages you feel like needing to fully explore the content!)

In [1]:
# if using google collab, run the following pip installs!
!pip install sobol_seq
!pip install plotly
!pip install gpytorch
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from scipy.integrate import quad
from scipy.spatial.distance import cdist
from scipy.optimize import minimize
import math
import time
import sobol_seq


## Section A: Introduction to Batch Bayesian Optimisation
A useful modification to the BO workflow is to allow for the acquisition functions to provide multiple/batch recommedations as chemical experiments can be performed in parallel. (Context: If you are an experimental chemist, think how many expeiments you can dispense, react, isolate and analyse in a given period vs consecutive singleton reactions.)

Although gaining speed from performing evaluations in parallel, the recommendations would be less informed, as batch recommendations are based on the conditioning of a smaller set of data when compared to singleton experiments. The suitable number of batch experiments is usually dependant on how many experiments can be run in parallel which is heavily reliant on the type of chemistry and the chemist who is runnning the experiment.

In this section, we will explore batch Bayesian Optimisation with Thompson Sampling.

#### Excersise A1_1:
##### Concepts: Run Time of Bayesian Optimisation

Apart form being able to perform experiments in parallel, batch BO can help speed up the runtime of the BO code. Using the GP and BO classes you have defined previously in Day 1, determine how long each section of the code (GP regression, GP inference, acquisition function etc.) takes to run. you can use the time module to help.

```python
time1 = time.time()
...
time2 = time.time()
print('Time taken in seconds: ', time2-time1)
```

Which section runs with constant time over each iteration? Which section takes longer time to run? Why is this? Compare the magnitude of the time taken for each section.

#### Excersise A1_2:
##### Concepts: Run Time of Bayesian Optimisation

Now increase your input dimensions by 1 (use an arbitrary objective function that takes in 3 inputs and returns 1 output). Keep the number of training data and the number of point per dimension the same. How long does it take to run each section of the code? How the the time taken scale with the number of dimensions - why is this so?



#### Excersise A1_3:
##### Concepts: Priors, Posteriors, Thompson Sampling

You should have observed that the acquisition function takes the least time to run and in several orders of magnitude quicker than the GP regression and inference. It could be useful to perform more evaluations per round of BO if the GP regression/inference requires a longer time, especially with higher dimensions.

We are going to observe an implimentation of batch BO using Thompson Sampling. But before this, we are going to first see GPs from a slightly different, but related angle. We have seen that the result of GP inference is that we obtain a mean and an associated variance of the search space is a result of conditioning on trainig data. Another view of the outcome of GP inference is that we obtain a **distribution of functions** that are conditioned on the trianing data of which gives the mean and associated variance over the interested search space.

We have made assumptions about the mean and kernel functions as seen in previous sections. Prior to conditioning, samples of this distribution can be plotted (with a given set of kernel hyperparameters).

![download.png](attachment:63cb9bc0-14b9-44fb-83d8-672dc148040a.png)

We can see that samples of the distribution of functions averages to the described mean function (here, = 0) with a constant variance (variance is not plotted here) over the entire search space. These functions are called the priors.

After conditioning, we can see the conditioned priors go through exactly the points of the training data (noise is not considered here). The average of these functions give the mean and provides the associated variance. The functions are called the posteriors.

![download.png](attachment:67478bcf-2c7b-4a14-8b79-eed4f3c97f2c.png)

We have seen previously that the acquisition function considered the mean and associated variance to obtain a single recommendation. Thompson Sampling allows us to obtain batch recommendations by considering individual posterior functions obtained from the distribution instead of the mean and variance directly. The number of recommendations (batch number) determines the number of sampled posteriors we require to obtain.

To obtain the posteriors, we need to obtain the mean and full covariance matrix of all points of the interested search space as each point in this space will have to be conditioned on every other point in this space. A function for doing so is created and seen below. It takes in an object of the GP_model_zeromean class and the interested search space as inputs and gives the mean and covariance matrix as output.

We can then draw posteriors using the `np.random.multivariate_normal()` function.

``` python
def GP_predict_full(gp, Xtest):
    """
    Returns (mean_vector, cov_matrix) at the inputs Xstar_norm.
    """
    Xstar_norm = (Xtest - gp.X_mean) / gp.X_std
    # unpack
    ell, sf2 = np.exp(2*gp.hyperparam_optimized[:gp.nx_dimensions, 0]), \
                np.exp(2*gp.hyperparam_optimized[gp.nx_dimensions, 0])
    sn2 = np.exp(2*gp.hyperparam_optimized[gp.nx_dimensions+1, 0]) + 1e-8
    
    # training data
    Xn, Yn = gp.X_norm, gp.Y_norm[:,0]         # here we only have one output
    K = gp.Cov_mat(gp.kernel, Xn, ell, sf2) + sn2*np.eye(gp.number_of_point)
    K = (K + K.T)*0.5
    Kinv = np.linalg.solve(K, np.eye(K.shape[0]))

    # cross-covariances
    # shape: (n_train, n_test)
    Ks = sf2 * np.exp(-0.5 * cdist(Xn, Xstar_norm, 'seuclidean', V=ell)**2)

    # test-test covariance
    Kss = sf2 * np.exp(-0.5 * cdist(Xstar_norm, Xstar_norm, 'seuclidean', V=ell)**2)

    # predictive mean and covariance
    mu_star = Ks.T.dot(Kinv).dot(Yn)
    cov_star = Kss - Ks.T.dot(Kinv).dot(Ks)
    # ensure symmetry
    cov_star = (cov_star + cov_star.T)*0.5

    #unnormalise the mean and covariance matrix
    mu_plot = mu_star*gp.Y_std + gp.Y_mean
    cov_plot = cov_star * (gp.Y_std**2)
    return  mu_plot, cov_plot

mu_plot, cov_plot = GP_predict_full(GP_m, X_searchspace.reshape(-1,1))
samples = np.random.multivariate_normal(mu_plot.flatten(), cov_plot, size=50) #drawing 50 posterior samples
plt.plot(X_searchspace.reshape(-1,1), samples[1], lw=0.5) #we can plot the posteriors, here the first of 50 posterior is plotted

```

Using this function, can you recreate the posterior plot as seen above? (We are not performing BO here, just GP regression, inference and obtaining posteriors.)

```python
def obj_func(x):
	return np.sin(x**np.sin(x))

# training data
X_training = np.array([3.5, 4, 6, 7])
ndata      = X_training.shape[0]
X_training = X_training.reshape(ndata,1)
Y_training = obj_func(X_training )

# search space
number_points_searchspace  = 500
X_searchspace              = np.linspace(3, 8.0, num=number_points_searchspace)

Ysearchspace_mean        = np.zeros(number_points_searchspace)
Ysearchspace_std         = np.zeros(number_points_searchspace)

GP_m = GP_model_meanzero(X_training, Y_training, 'SquaredExponential', hyperparams_multistart_loops = 3)

for number in range(len(X_searchspace)):
    m_ii, std_ii   = GP_m.GP_inference_np(X_searchspace[number])
    Ysearchspace_mean[number] = m_ii.item()
    Ysearchspace_std[number]  = std_ii.item()


ax = plt.figure(figsize = (8,4), dpi = 100)
plt.plot(X_training, Y_training  , 'kx', mew=2)
plt.plot(X_searchspace,Ysearchspace_mean, 'C0', lw=2)
plt.gca().fill_between(X_searchspace.flat,
                       Ysearchspace_mean  - 3*np.sqrt(Ysearchspace_std),
                       Ysearchspace_mean  + 3*np.sqrt(Ysearchspace_std),
                       color='C0', alpha=0.2)

#####
#Obtain full mean and covariance matrix of interested search space, obtain posteriors and plot
#Write your code here!
#####

plt.title('Gaussian Process Regression')
plt.legend(('Training Data', 'Objective Function', 'GP mean', 'GP Confidence Interval'),
           loc='lower right')
plt.show()
```

In [3]:
# You can use your own GP class or use the class below
# If using your own GP class - remember to modify the ...
# 'GP_m = GP_model_meanzero(X_training, Y_training, 'SquaredExponential', hyperparams_multistart_loops = 3)'
# ... section of the above code! (And do any other troubleshooting - hence would recommend using the code below for ease')

class GP_model_meanzero:

    def __init__(self, X, Y, kernel, hyperparams_multistart_loops):
        self.X, self.Y, self.kernel = X, Y, kernel
        self.number_of_point, self.nx_dimensions, self.ny_dimensions = X.shape[0], X.shape[1], Y.shape[1]
        self.multistart_loops            = hyperparams_multistart_loops

        #Normalisation
        self.X_mean, self.X_std     = np.mean(X, axis=0), np.std(X, axis=0)
        self.Y_mean, self.Y_std     = np.mean(Y, axis=0), np.std(Y, axis=0)
        self.X_norm, self.Y_norm    = (X-self.X_mean)/self.X_std, (Y-self.Y_mean)/self.Y_std

        #Determine Kernel Hyperparameters
        self.hyperparam_optimized , self.inverse_covariance_matrix_opt   = self.determine_hyperparameters()

    def Cov_mat(self, kernel, X_norm, W, sf2):
        if kernel == 'SquaredExponential':
            xixj_euclidean_distance = cdist(X_norm, X_norm, 'seuclidean', V=W)**2
            cov_matrix = sf2*np.exp(-0.5*xixj_euclidean_distance)
            return (cov_matrix)
        else:
            print('ERROR no kernel with name ', kernel)


    def negative_loglikelihood(self, hyper, X, Y):
        # internal parameters
        n_point, nx_dim = self.number_of_point, self.nx_dimensions
        kernel          = self.kernel

        W               = np.exp(2*hyper[:nx_dim])   # W <=> 1/lambda
        sf2             = np.exp(2*hyper[nx_dim])    # variance of the signal
        sn2             = np.exp(2*hyper[nx_dim+1])  # variance of noise

        # obtaining negative logliklihood via Cholesky decomposition
        K       = self.Cov_mat(kernel, X, W, sf2)
        K       = K + (sn2 + 1e-8)*np.eye(n_point)
        K       = (K + K.T)*0.5
        L       = np.linalg.cholesky(K)
        logdetK = 2 * np.sum(np.log(np.diag(L)))
        invLY   = np.linalg.solve(L,Y)
        alpha   = np.linalg.solve(L.T,invLY)
        NLL     = np.dot(Y.T,alpha) + logdetK
        return (NLL)


    def determine_hyperparameters(self):
        # setting up bounds for the log likelyhood minimsation
        lower_bound = np.array([-4.]*(self.nx_dimensions+1) + [-8.])  # lengthscales + signal variance, noise variance
        upper_bound = np.array([4.]*(self.nx_dimensions+1) + [ -2.])
        bounds      = np.hstack((lower_bound.reshape(self.nx_dimensions+2,1), upper_bound.reshape(self.nx_dimensions+2,1)))

        #gives number of input set of random starting guesses for each hyperparameter
        multi_startvec         = sobol_seq.i4_sobol_generate(self.nx_dimensions + 2, self.multistart_loops)

        #variables for storing information during loop
        temp_min_hyperparams   = [0.]*self.multistart_loops
        temp_loglikelihood     = np.zeros((self.multistart_loops))
        hyperparam_optimized   = np.zeros((self.nx_dimensions+2, self.ny_dimensions)) #for best solutions
        inverse_covariance_matrix_opt = []

        #minimisation of hyperparameters
        for i in range(self.ny_dimensions):
            for j in range(self.multistart_loops ):
                #initilising hyperparam guess
                hyperparams_initialisation   = lower_bound + (upper_bound-lower_bound)*multi_startvec[j,:] # mapping sobol unit cube to boudns

                result = minimize(self.negative_loglikelihood,
                               hyperparams_initialisation,
                               args=(self.X_norm, self.Y_norm[:,i]),
                               method='SLSQP',
                               options={'disp':False,'maxiter':10000},
                               bounds=bounds,
                               tol=1e-12)

                temp_min_hyperparams[j] = result.x
                temp_loglikelihood[j]   = result.fun

            # choosing best solution from temporary lists
            minimumloglikelihood_index    = np.argmin(temp_loglikelihood)
            hyperparam_optimized[:,i]     = temp_min_hyperparams[minimumloglikelihood_index  ]

            # exponential to recover value from log space
            lengthscale_opt         = np.exp(2.*hyperparam_optimized[:self.nx_dimensions,i])
            signalvarience_opt      = np.exp(2.*hyperparam_optimized[self.nx_dimensions,i])
            noise_opt               = np.exp(2.*hyperparam_optimized[self.nx_dimensions+1,i]) + 1e-8

            #obtain convarience matrix from optimised kernel hyper parameters
            covarience_matrix_opt              = self.Cov_mat(self.kernel, self.X_norm, lengthscale_opt,signalvarience_opt) + noise_opt*np.eye(self.number_of_point)
            self.covarience_matrix_opt         = covarience_matrix_opt
            inverse_covariance_matrix_opt     += [np.linalg.solve(covarience_matrix_opt, np.eye(self.number_of_point))]

        return (hyperparam_optimized , inverse_covariance_matrix_opt)

    def calc_cov_sample(self,xnorm,Xnorm,ell,sf2):
        # internal parameters
        nx_dim = self.nx_dimensions

        #covariance of sample
        dist = cdist(Xnorm, xnorm.reshape(1,nx_dim), 'seuclidean', V=ell)**2
        cov_matrix = sf2 * np.exp(-.5*dist)
        return (cov_matrix )


    def GP_inference_np(self, x):
        nx_dim                   = self.nx_dimensions
        kernel, ny_dim           = self.kernel, self.ny_dimensions
        hypopt, Cov_mat          = self.hyperparam_optimized, self.Cov_mat
        stdX, stdY, meanX, meanY = self.X_std, self.Y_std, self.X_mean, self.Y_mean
        calc_cov_sample          = self.calc_cov_sample
        invKsample               = self.inverse_covariance_matrix_opt
        Xsample, Ysample         = self.X_norm, self.Y_norm

        xnorm = (x - meanX)/stdX
        mean  = np.zeros(ny_dim)
        var   = np.zeros(ny_dim)

        # ny_dim -> number of outputs
        for i in range(ny_dim):
            invK           = invKsample[i]
            hyper          = hypopt[:,i]
            ellopt, sf2opt = np.exp(2*hyper[:nx_dim]), np.exp(2*hyper[nx_dim])

            # calculation of covaraince for each output
            # although noise hyperparameter determiend, it is ignored here for better visualisation
            k       = calc_cov_sample(xnorm,Xsample,ellopt,sf2opt)
            mean[i] = np.matmul(np.matmul(k.T,invK),Ysample[:,i])
            var[i]  = max(0, sf2opt - np.matmul(np.matmul(k.T,invK),k))

        # un-normalisation (obtianing actual mean and variance)
        mean_sample = mean*stdY + meanY
        var_sample  = var*stdY**2

        return (mean_sample, var_sample)

#### Excersise A1_4:
##### Concepts: Thompson Sampling, Batch BO

Now that we can draw samples of the posterior, we can now adapt the `BO_1D` class below to allow for batch BO!

Similar to the LCB acquisition function, we can draw recommendations from taking the minima (to perform minimisation) from each of the sampled posteriors. Can you modify the `BO_1D` class with the following to allow for batch BO?

1. Add a new 'batch' input to specify the number of recommendations per iteration
2. Create a new 'ThompsonSamnpling' acquisition function which draws the number of posterior sample specified by the 'batch_number' and obtain the input that correspond with the posterior minimum. `np.argmin()` function can be useful. It returns the index of the minimum number form a list.
3. Modify the `plt.plot(range(iterations), self.exploredY)` function to a scatter plot to observe the batch evaluations over the iterations.

```python
class BO_1D:
    def __init__(self, X, kernel, X_searchspace,  iterations, acquisition_function, objective_func, print_graph, acquisition_hyperparam):       
        
        self.X = X

        Fx_training              = np.array([objective_func(x) for x in self.X])
        self.Y                   = Fx_training.reshape(Fx_training.shape[0],1)
        
        fx_searchspace           = np.array([objective_func(x) for x in X_searchspace])
        Ysearchspace_mean        = np.zeros(number_points_searchspace**(np.shape(X_training)[1]))
        Ysearchspace_std         = np.zeros(number_points_searchspace**(np.shape(X_training)[1]))
        
        self.minY = []
        self.exploredY = []
        
        for i in range(iterations):
            GP_m = GP.GP_model_meanzero(self.X, self.Y, kernel, hyperparams_multistart_loops = 3)
            
            for number in range(len(X_searchspace)):
                m_ii, std_ii   = GP_m.GP_inference_np(X_searchspace[number])
                Ysearchspace_mean[number] = m_ii.item()
                Ysearchspace_std[number]  = std_ii.item()
            
            if acquisition_function == 'Thompson':
               ... # Add thompson sampling code here!
                
            else:
                print('No acquisition function called ', acquisition_function)
                break
            
            self.X = np.append(self.X, [[X_acquisitionfunc]],0)
            self.Y = np.append(self.Y, [[objective_func(X_acquisitionfunc)]],0)

            self.minY += [min(self.Y)]
            self.exploredY += [objective_func(X_acquisitionfunc)]

            if print_graph == True:
                pass
            else:
                pass
        plt.figure(figsize = (8,4), dpi = 100)
        plt.title('Minimum of Training Data set over Iterations')
        plt.xlabel('Iterations')
        plt.plot(range(iterations), self.minY)
        plt.show()
        plt.figure(figsize = (8,4), dpi = 100)
        plt.title('Evaluation Output over Iterations')
        plt.xlabel('Iterations')
        plt.plot(range(iterations), self.exploredY)
        plt.show()

```

#### Excersise A1_5:
##### Concepts: Thompson Sampling, Batch BO

A code for the same implimentation of batch BO with thompson sampling is given below. We will run BO in 2 and 3 dimensions with the inputs specified below. Observe how much time it will take for only 2 rounds of BO with only 20 points for each input dimensions! Why does it take so much longer when going to higher dimension?

*warning: 3 dimension computation could take a few minutes

``` python
class BO_multiD_batch:
    def __init__(self, X, kernel, X_searchspace, iterations,acquisition_function, objective_func,print_graph, acquisition_hyperparam, batch):

        # 1) initilisation
        self.X = np.array(X)
        # 2) build Y as an (n×1) vector
        Fx_training = np.array([objective_func(x) for x in self.X])
        self.Y = Fx_training.reshape(-1, 1)

        self.minY = []
        self.exploredY = []

        for i in range(iterations):
            # 3) fit GP to the current (X,Y)
            GP_m = GP_model_meanzero(self.X, self.Y, kernel, hyperparams_multistart_loops=3)

            # 4) predict over the search space
            means = np.zeros(len(X_searchspace))
            vars  = np.zeros(len(X_searchspace))
            for idx, xx in enumerate(X_searchspace):
                m, v = GP_m.GP_inference_np(xx)
                means[idx] = m.item()
                vars[idx]  = v.item()

            # 5) sample from the joint posterior for Thompson sampling
            mu_plot, cov_plot = GP_predict_full(GP_m, X_searchspace)
            samples = np.random.multivariate_normal(mu_plot, cov_plot, size=batch)

            # 6) pick batch points
            new_X = []
            for s in samples:
                choice = np.argmin(s)
                new_X.append(X_searchspace[choice])
            new_X = np.array(new_X)

            # 7) evaluate objective and append to X,Y
            new_Y = np.array([objective_func(x) for x in new_X]).reshape(-1, 1)
            self.X = np.vstack([self.X, new_X])
            self.Y = np.vstack([self.Y, new_Y])

            self.minY     += [self.Y.min()]
            self.exploredY += [new_Y.flatten().tolist()]

            if print_graph:
                print(f"Iteration {i} complete.")

        # 8) finally, plot
        plt.figure(figsize=(8,4), dpi=100)
        plt.title('Minimum of Training Data set over Iterations')
        plt.xlabel('Iterations')
        plt.plot(range(iterations), self.minY)
        plt.show()

        plt.figure(figsize=(8,4), dpi=100)
        plt.title('Evaluation Output over Iterations')
        plt.xlabel('Iterations')
        for i, vals in enumerate(self.exploredY):
            plt.scatter([i]*batch, vals)
        plt.show()
```

2 dimension input:

``` python
x1loc  = [-4,  -1,  2]; x2loc = [-3, -2, 7]
Xtrain = [[x,y] for x in x1loc for y in x2loc]
X_training = np.array(Xtrain)

number_points_pervariable      = 20
number_points_searchspace   = number_points_pervariable ** (np.shape(X_training)[1])

#Here, the interested search space is symmetrical, if it is non symmetrical, one can define each dimension individually and build the search space recursively
X_searchspace     = np.linspace(-5, 8, num=number_points_pervariable)
X_searchspace     = np.array([[x,y] for x in X_searchspace for y in X_searchspace])

def obj_func(X):
	return (np.sin(X[0])+np.sin(X[1]))

BO_m = BO_multiD_batch(X = X_training,  
   kernel = 'SquaredExponential',
   X_searchspace = X_searchspace,
   iterations = 2,
   acquisition_function = 'Thompson',
   objective_func = obj_func,
   print_graph = True,
   acquisition_hyperparam=[10],
    batch = 5)

```

3 dimension input:

``` python
x1loc  = [-4,  -1]; x2loc = [-3, -2]; x3loc = [-3, 1]
Xtrain = [[x,y,a] for x in x1loc for y in x2loc for a in x3loc]
X_training = np.array(Xtrain)

number_points_pervariable    = 20
number_points_searchspace   = number_points_pervariable ** (np.shape(X_training)[1])

#Here, the interested search space is symmetrical, if it is non symmetrical, one can define each dimension individually and build the search space recursively
X_searchspace     = np.linspace(-5, 8, num=number_points_pervariable)
X_searchspace     = np.array([[x,y,a] for x in X_searchspace for y in X_searchspace for a in X_searchspace])

def obj_func(X):
	return (np.sin(X[0])+np.sin(X[1])+np.cos(X[2]))

BO_m = BO_multiD_batch(X = X_training,  
   kernel = 'SquaredExponential',
   X_searchspace = X_searchspace,
   iterations = 2,
   acquisition_function = 'Thompson',
   objective_func = obj_func,
   print_graph = True,
   acquisition_hyperparam=[10],
    batch = 5)
```

#### Excersise A1_6:
##### Concepts: Marginal Thompson Sampling, Batch BO

We can attibute the long run times to the need for the full covariance matrix to be evaluated in `GP_predict_full()`. Another method to obtain batach recommendations is through a tecnique called 'Marginal Thompson Sampling'. Here, we are not obtaining posterior samples. Instead, we can draw individual samples of each point with the mean and standard deviation of each point (no covariance required) with the standard deviation scaled by a hyperparameter (similar to LCB) and a scalar drawn from the standard normal distribution via `np.random.randn()`. We can observe and example of 5 batch samples and recommendations in the following plot.

![download.png](attachment:6da18bad-6429-4a02-8f05-a206b96cfbd5.png)

An implimentation to obtain Marginal TS samples is given below. Similar to excersise A1_4, can you modify your previous `BO_1D_batch` function to perform batch BO via marginal TS? (You do not have to use the given code if it does not fit with your previous implimentation.)

``` python

for number in range(batch):
    if acquisition_function == 'ThompsonMarginal':
        ts_sample  = Ysearchspace_mean + acquisition_hyperparam[0]*np.sqrt(Ysearchspace_std)*np.random.randn(X_searchspace.shape[0])
        new_X.append(X_searchspace[np.argmin(ts_sample)])
        plt.plot(X_searchspace,ts_sample, lw = 0.2, alpha = 0.3)
    else:
        print('No acquisition function called ', acquisition_function)
        break

```


Use the following code to test your batch BO code.

``` python
def obj_func(x):
	return np.sin(x**np.sin(x))

# training data
X_training = np.array([3.5, 4])
ndata      = X_training.shape[0]
X_training = X_training.reshape(ndata,1)

# search space
number_points_searchspace  = 500
X_searchspace              = np.linspace(3, 8.0, num=number_points_searchspace)

BO_m = BO_1D_batch(X = X_training,  
   kernel = 'SquaredExponential',
   X_searchspace = X_searchspace,
   iterations = 25,
   acquisition_function = 'ThompsonMarginal',
   objective_func = obj_func,
   print_graph = True,
   acquisition_hyperparam=[3],
   batch = 5)
```

#### Excersise A1_7:
##### Concepts: Marginal Thompson Sampling, Batch BO

Similar to excersise A1_5, some code for the multidimensional batch BO is given below. Use this code and explore how one could perform multidimensional BO (go up to 5/6 dimensions with your chosen objective function! See how the various parts of the inputs (hyperparameters etc.) and code perform with varying values! How long does the code take to run!). An example with a 2D input is shown.

```python
class BO_multiD_batch:
    def __init__(self, X, kernel, X_searchspace, iterations,acquisition_function, objective_func,print_graph, acquisition_hyperparam, batch):

        # 1) initilisation
        self.X = np.array(X)
        # 2) build Y as an (n×1) vector
        Fx_training = np.array([objective_func(x) for x in self.X])
        self.Y = Fx_training.reshape(-1, 1)

        self.minY = []
        self.exploredY = []

        for i in range(iterations):
            # 3) fit GP to the current (X,Y)
            GP_m = GP_model_meanzero(self.X, self.Y, kernel, hyperparams_multistart_loops=3)

            # 4) predict over the search space
            means = np.zeros(len(X_searchspace))
            varience  = np.zeros(len(X_searchspace))
            for idx, xx in enumerate(X_searchspace):
                m, v = GP_m.GP_inference_np(xx)
                means[idx] = m.item()
                varience[idx]  = v.item()

            # 5) sample from the joint posterior for Thompson sampling
            if acquisition_function == 'ThompsonMarginal':
                new_X = []
                for number in range(batch):
                    ts_sample  = means + acquisition_hyperparam[0]*np.sqrt(varience)*np.random.randn(X_searchspace.shape[0])
                    new_X.append(X_searchspace[np.argmin(ts_sample)])
                new_X = np.array(new_X)
            else:
                print('No acquisition function named', acquisition_function)
            
            # 7) evaluate objective and append to X,Y
            new_Y = np.array([objective_func(x) for x in new_X]).reshape(-1, 1)
            self.X = np.vstack([self.X, new_X])
            self.Y = np.vstack([self.Y, new_Y])

            self.minY     += [self.Y.min()]
            self.exploredY += [new_Y.flatten().tolist()]

            if print_graph:
                print(f"Iteration {i} complete.")

        # 8) finally, plot
        plt.figure(figsize=(8,4), dpi=100)
        plt.title('Minimum of Training Data set over Iterations')
        plt.xlabel('Iterations')
        plt.plot(range(iterations), self.minY)
        plt.show()

        plt.figure(figsize=(8,4), dpi=100)
        plt.title('Evaluation Output over Iterations')
        plt.xlabel('Iterations')
        for i, vals in enumerate(self.exploredY):
            plt.scatter([i]*batch, vals)
        plt.show()

```

For 2D input:

``` python
x1loc  = [-4,  -1,  2]; x2loc = [-3, -2, 7]
Xtrain = [[x,y] for x in x1loc for y in x2loc]
X_training = np.array(Xtrain)

number_points_pervariable      = 20
number_points_searchspace   = number_points_pervariable ** (np.shape(X_training)[1])

#Here, the interested search space is symmetrical, if it is non symmetrical, one can define each dimension individually and build the search space recursively
X_searchspace     = np.linspace(-5, 8, num=number_points_pervariable)
X_searchspace     = np.array([[x,y] for x in X_searchspace for y in X_searchspace])

def obj_func(X):
	return (np.sin(X[0])+np.sin(X[1]))

BO_m = BO_multiD_batch(X = X_training,  
   kernel = 'SquaredExponential',
   X_searchspace = X_searchspace,
   iterations = 20,
   acquisition_function = 'ThompsonMarginal',
   objective_func = obj_func,
   print_graph = True,
   acquisition_hyperparam=[10],
   batch = 5)
```



## Section B: Introduction to Reaction Optimisation with BO

We are now going to use our batch BO workflow to run for an optimisation campaign for a chemical reaction! The case study can be found in this reference.

![image.png](attachment:71fb980f-79e0-4f55-96be-9c020a0d5589.png)

Alexander Buitrago Santanilla et al. ,Nanomole-scale high-throughput chemistry for the synthesis of complex molecules.Science347,49-53(2015).DOI:<a href="https://www.science.org/doi/10.1126/science.1259203">10.1126/science.1259203</a>

The reaction condition space for a particular chemical transformation can be extremely vast due to the combinatorial nature of variables (especially if the reaction is not established). For example, our case study is a Buchwald-Hartwig amination (C-N cross coupling) which requires a palladium catalyst, ligand, base and solvent. Just considering 10 chemicals for each reagent will lead to a space of 10000 possible reaction conditions!

Bayesian Optimisation provides a systematic, data-driven way of choosing which experiments to perform. We will only be considering 2 variables in the optimisation of this reaction: The pre-catalyst and base. Our goal is to obtain the reaction condition with the highest yield (which combination of base and pre-catlayst gives the highest yield?). Below is the list of pre-catalyst and bases we will be examining.

A word about the pre-catalyst: Pre-catalysts are precursors to the active catalyst for a catalystic reaction. They are often have the catalytic center (the metal atom) and the ligand pre-bonded such that the pre-catalyst can be directly used or dispense for a reaction. There are 4 generations to the palladium pre-catalyst. This case study uses the 2nd and 3rd generation of precatalyst. The structures of the pre-catlaysts, ligands and bases are seen below (L = ligand).

![image.png](attachment:03e13af8-5308-432e-8e13-693dd531c9f4.png)
![image.png](attachment:75f349bd-c554-4793-bc70-d021d02ff3fd.png)

In [ ]:
precatalyst = ['G3_BINAP',
               'G3_DPPF',
               'G2_XantPhos',
               'G2_tertBu3P',
               'G3_PPA',
               'G3_Aphos',
               'G3_Xphos',
               'G2_RuPhos',
               'G3_DTBPF',
               'G3_J009',
               'G3_MorDalPhos',
               'G3_BrettPhos',
               'G3_tertBuXPhos',
               'G3_tertBuBrettPhos',
               'G3_RockPhos',
               'G3_AdBrettPhos']
base = ['DBU',
        'MTBD',
        'BTMG',
        'BEMP',
        'BTTP',
        'P2Et']

#### Excersise B1_1:
##### Concepts: Chemical Descriptors

Before perfoming BO, we have to convert the pre-catalysts and bases into a machine-readable form. GP regressors are not able to process each reagent just from the names. We have to translate the chemical identities into a numerical number such that the GP can not only interpret them but also draw relations between them (via the mean and covariance matrix). There are many ways one can generate descriptor for each molecule.

A few include the use of density functional theory descriptors, cheminformatics descriptors, fingerprinting, experimental values etc. There are whole databases dedicated to the generation and storage of descriptors. <a href="https://descriptor-libraries.molssi.org/kraken/">An example is the KRAKEN data base for ligands. </a>

The best descriptors are ones which directly relate to the outcome of the reaction. For example, if a base of higher pKaH value leads to a higher yield, then pKaH would be a good descriptor. However, it is rare to know prior to the optimisation of the reaction (or similar reactions) if a descriptors is suitable. There are many examples of chemical optimisation of BO that take the dimensionally reduced components of a very long list of descriptors.

However, for demonstration purposes, we will stick to some simple descriptors such as molecular mass for the pre-catalyst and pKaH for the bases.

```python
precatalyst_descriptors = [['G3_BINAP',[622.20]],
                           ['G3_DPPF',[544.10]],
                           ['G2_XantPhos',[578.19]],
                           ['G2_tertBu3P',[202.19]],
                           ['G3_PPA',[292.12]],
                           ['G3_Aphos',[265.20]],
                           ['G3_Xphos',[476.36]],
                           ['G2_RuPhos',[466.30]],
                           ['G3_DTBPF',[474.23]],
                           ['G3_J009',[554.29]],
                           ['G3_MorDalPhos',[463.30]],
                           ['G3_BrettPhos',[536.38]],
                           ['G3_tertBuXPhos',[424.33]],
                           ['G3_tertBuBrettPhos',[484.35]],
                           ['G3_RockPhos',[468.35]],
                           ['G3_AdBrettPhos',[640.44]]]

base_descriptors = [['DBU', [24.3]],
                    ['MTBD', [25.5]],
                    ['BTMG', [23.6]],
                    ['BEMP', [27.6]],
                    ['BTTP', [28.4]],
                    ['P2Et', [32.9]]]
```

Can you find some alternative descriptors for the precatalyst and base? We will used this to perform BO and evaluate which set of descriptors are better.

#### Excersise B1_2:
##### Concepts: Optimisation of Chemical Reactions with BO

We can now perform some BO! Usually, the evaluation of the reaction condition will involve experimentation. In this case, an objective function (with a delay in the BO class) will be used to emulate experimentation. This objective function takes in the descriptors and searches in the search space for the index with matching descriptors and returns the yield as a function of the index.

It is therefore very important that the search space is built in the same order as the yield list! (the notebook will explicitly provide the order in which the search space is to be built.

Since we have built a BO algorithm to minimise, the yields are inverted into the negative space such that minimisation will conrespond to obtaining the highest yield.

```python
reaction_yield = [-3.6, -3.9, -3.5, -3.3, -3.6, -3.0, -1.1, -0.7, -0.8, -0.6, -0.8, -1.2, -0.7, -1.5, -0.0, -0.0, -0.0, -0.5, -5.0, -0.0, -0.0, -1.6, -3.5, -6.6, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.5, -1.5, -0.0, -0.0, -0.0, -0.7, -0.0, -1.6, -0.0, -1.3, -2.2, -23.2, -0.0, -0.0, -0.0, -1.9, -4.0, -16.3, -4.0, -1.0, -0.0, -0.0, -0.0, -62.7, -0.4, -1.8, -1.7, -1.0, -0.0, -0.0, -0.1, -0.3, -0.3, -0.4, -0.4, -2.8, -1.1, -1.4, -2.6, -0.7, -1.4, -2.1, -10.9, -91.7, -100.0, -50.2, -27.8, -79.1, -0.0, -3.1, -0.0, -1.2, -1.8, -14.5, -0.5, -0.5, -0.0, -0.4, -0.6, -7.8, -3.1, -3.5, -3.2, -2.2, -3.0, -15.8]

def experiment_yield(condition , X_searchspace):
    return(reaction_yield[X_searchspace.index(condition)])

#ordering of the search space is very important
X_searchspace = []
for precat in precatalyst_descriptors:
    for base in base_descriptors:
        X_searchspace += [precat[1]+base[1]]

```

Code for batch BO using marginal Thompson Sampling and Squared Exponential kernel functions is observed below.

```python
class BO_multiD_batch_chemistry:
    def __init__(self, X, kernel, X_searchspace, iterations,acquisition_function, objective_func,print_graph, acquisition_hyperparam, batch, experiment_time):

        self.X = np.array(X)
        Fx_training = np.array([objective_func(x.tolist(),X_searchspace) for x in self.X])
        self.Y = Fx_training.reshape(-1, 1)
        
        self.minY = []
        self.exploredY = []

        for i in range(iterations):
            GP_m = GP_model_meanzero(self.X, self.Y, kernel, hyperparams_multistart_loops=3)

            means = np.zeros(len(X_searchspace))
            varience  = np.zeros(len(X_searchspace))
            for idx, xx in enumerate(X_searchspace):
                m, v = GP_m.GP_inference_np(xx)
                means[idx] = m.item()
                varience[idx]  = v.item()

            
            if acquisition_function == 'ThompsonMarginal':
                new_X = []
                for number in range(batch):
                    ts_sample  = means + acquisition_hyperparam[0]*np.sqrt(varience)*np.random.randn(len(X_searchspace))
                    new_X.append(X_searchspace[np.argmin(ts_sample)])
                new_X = np.array(new_X)
            else:
                print('No acquisition function named', acquisition_function)

            time.sleep(experiment_time)
            new_Y = np.array([objective_func(x.tolist(),X_searchspace) for x in new_X]).reshape(-1, 1)
            self.X = np.vstack([self.X, new_X])
            self.Y = np.vstack([self.Y, new_Y])

            self.minY     += [self.Y.min()]
            self.exploredY += [new_Y.flatten().tolist()]

            if print_graph:
                print(f"Iteration {i} complete.")

        plt.figure(figsize=(8,4), dpi=100)
        plt.title('Minimum of Training Data set over Iterations')
        plt.xlabel('Iterations')
        plt.plot(range(iterations), self.minY)
        plt.show()

        plt.figure(figsize=(8,4), dpi=100)
        plt.title('Evaluation Output over Iterations')
        plt.xlabel('Iterations')
        for i, vals in enumerate(self.exploredY):
            plt.scatter([i]*batch, vals)
        plt.show()

BO_m = BO_multiD_batch_chemistry(X = X_training,  
                                 kernel = 'SquaredExponential',
                                 X_searchspace = X_searchspace,
                                 iterations = 15,
                                 acquisition_function = 'ThompsonMarginal',
                                 objective_func = experiment_yield,
                                 print_graph = True,
                                 acquisition_hyperparam=[3],
                                 batch = 5,
                                 experiment_time = 0)

```

Using this code, start your first reaction optimisation campaign using BO! (We can leave the experiment_time to 0 for now.) Start with 3 training data points - these must be in the correct order of precatayst descriptors first then base descriptors. For example:

```python
X_training = [[544.1, 32.9],[265.2, 24.3],[640.44, 25.5]]
```

#### Excersise B1_3:
##### Concepts: Optimisation of Chemical Reactions with BO

We can also use multidimensional desciptors for each molecule! This is usually beneficial especially when there are multiple factors that contribute towards the goal. (For example, when sterics and electronic properties of the molecule affects the outcome). However, as seen previously, the more dimensions, the longer the time the BO will take. For now, lets observe an example of a 2D input for the precatalyst.

With higher dimensions, it is required that the training data include at least 2 different data points for the same dimension. Otherwise you will get an error (the covariance matrix cannot be formed).

```python
precatalyst_descriptors = [['G3_BINAP',[622.20, 2]],
                           ['G3_DPPF',[544.10, 2]],
                           ['G2_XantPhos',[578.19, 2]],
                           ['G2_tertBu3P',[202.19, 1]],
                           ['G3_PPA',[292.12, 1]],
                           ['G3_Aphos',[265.20, 1]],
                           ['G3_Xphos',[476.36, 1]],
                           ['G2_RuPhos',[466.30, 1]],
                           ['G3_DTBPF',[474.23, 2]],
                           ['G3_J009',[554.29, 2]],
                           ['G3_MorDalPhos',[463.30, 1]],
                           ['G3_BrettPhos',[536.38, 1]],
                           ['G3_tertBuXPhos',[424.33, 1]],
                           ['G3_tertBuBrettPhos',[484.35, 1]],
                           ['G3_RockPhos',[468.35, 1]],
                           ['G3_AdBrettPhos',[640.44, 1]]]

base_descriptors = [['DBU', [24.3]],
                    ['MTBD', [25.5]],
                    ['BTMG', [23.6]],
                    ['BEMP', [27.6]],
                    ['BTTP', [28.4]],
                    ['P2Et', [32.9]]]

X_training = [[544.1, 2, 32.9],[265.2, 1, 24.3],[640.44, 1, 25.5], [622.20, 2, 28.4]]
X_searchspace = []
for precat in precatalyst_descriptors:
    for base in base_descriptors:
        X_searchspace += [precat[1]+base[1]]

BO_m = BO_multiD_batch_chemistry(X = X_training,  
   kernel = 'SquaredExponential',
   X_searchspace = X_searchspace,
   iterations = 15,
   acquisition_function = 'ThompsonMarginal',
   objective_func = experiment_yield,
   print_graph = True,
   acquisition_hyperparam=[3],
   batch = 5,
   experiment_time=0)
```

#### Excersise B1_4:
##### Concepts: Optimisation of Chemical Reactions with BO

Well done! Now that you have succesfully run a BO optimisation campaign and known the various aspects of BO. Please explore how changing various aspects of the input (hyperparameters, descriptors, iterations vs batch size for a given budget etc.), changing the code (a different acquisition function etc.) affect the performance and runtime of the code.

This will greatly help you in the coming hackathon!